# Power grid example model definition
We considere the following power grid taken from [a CIGRE network](http://www.cigre.org/News/Network-of-the-Future) described by the following picture

<img src="https://pandapower.readthedocs.io/en/v1.4.1/_images/cigre_network_lv.png" alt="Model" style="width: 500px;"/>

A model of this network can be created using pandapower as follow:

In [1]:
import pandapower as pp
 
net = pp.create_empty_network()

/home/pablo/Projects/zerobnl/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Create lines types from parameters

In [2]:
line_data = {"c_nf_per_km": 0.0, "r_ohm_per_km": 0.4917, "x_ohm_per_km": 0.2847, "max_i_ka": 1.0, "type": "ol"}
pp.create_std_type(net, line_data, "OH1", element='line')

In [3]:
line_data = {"c_nf_per_km": 0.0, "r_ohm_per_km": 1.3207, "x_ohm_per_km": 0.3210, "max_i_ka": 1.0, "type": "ol"}
pp.create_std_type(net, line_data, "OH2", element='line')

In [4]:
line_data = {"c_nf_per_km": 0.0, "r_ohm_per_km": 0.1620, "x_ohm_per_km": 0.0832, "max_i_ka": 1.0, "type": "cs"}
pp.create_std_type(net, line_data, "UG1", element='line')

In [5]:
line_data = {"c_nf_per_km": 0.0, "r_ohm_per_km": 0.2647, "x_ohm_per_km": 0.0823, "max_i_ka": 1.0, "type": "cs"}
pp.create_std_type(net, line_data, "UG2", element='line')

In [6]:
line_data = {"c_nf_per_km": 0.0, "r_ohm_per_km": 0.8220, "x_ohm_per_km": 0.0847, "max_i_ka": 1.0, "type": "cs"}
pp.create_std_type(net, line_data, "UG3", element='line')

## Define geographical information
Upload geodata for buses from .csv

In [7]:
import pandas as pd

geo = pd.read_csv("bus_geodata.csv", index_col=0)

## Implement model
Create buses (MV and LV)

In [8]:
coord = (geo.loc["Bus 0 MV", "x"], geo.loc["Bus 0 MV", "y"])
pp.create_bus(net, name="Bus 0 MV", vn_kv=20.0, type="b", geodata=coord)

coord = (geo.loc["Bus R0 MV", "x"], geo.loc["Bus R0 MV", "y"])
pp.create_bus(net, name="Bus R0 MV", vn_kv=20.0, type="b", geodata=coord)
for i in range(1, 19):
    name = "Bus R{} LV".format(i)
    coord = (geo.loc[name, "x"], geo.loc[name, "y"])
    pp.create_bus(net, name=name, vn_kv=0.4, type="n", geodata=coord)

coord = (geo.loc["Bus I0 MV", "x"], geo.loc["Bus I0 MV", "y"])
pp.create_bus(net, name="Bus I0 MV", vn_kv=20.0, type="b", geodata=coord)
for i in range(1, 3):
    name = "Bus I{} LV".format(i)
    coord = (geo.loc[name, "x"], geo.loc[name, "y"])
    pp.create_bus(net, name=name, vn_kv=0.4, type="n", geodata=coord)

coord = (geo.loc["Bus C0 MV", "x"], geo.loc["Bus C0 MV", "y"])
pp.create_bus(net, name="Bus C0 MV", vn_kv=20.0, type="b", geodata=coord)
for i in range(1, 21):
    name = "Bus C{} LV".format(i)
    coord = (geo.loc[name, "x"], geo.loc[name, "y"])
    pp.create_bus(net, name=name, vn_kv=0.4, type="n", geodata=coord)

Add lines (LV) -> Residential

In [9]:
for i in range(2, 11):
    from_bus = pp.get_element_index(net, "bus", "Bus R{} LV".format(i-1))
    to_bus = pp.get_element_index(net, "bus", "Bus R{} LV".format(i))
    pp.create_line(net, from_bus, to_bus, length_km=0.035, std_type="UG1", name="Line R{}-R{}".format(i-1, i))

for u, v in [(3, 11), (4, 12), (12, 13), (13, 14), (14, 15), (6, 16), (9, 17), (10, 18)]:
    from_bus = pp.get_element_index(net, "bus", "Bus R{} LV".format(u))
    to_bus = pp.get_element_index(net, "bus", "Bus R{} LV".format(v))
    pp.create_line(net, from_bus, to_bus, length_km=0.030, std_type="UG3", name="Line R{}-R{}".format(u, v))

Add lines (LV) -> Industral

In [10]:
from_bus = pp.get_element_index(net, "bus", "Bus I1 LV")
to_bus = pp.get_element_index(net, "bus", "Bus I2 LV")
pp.create_line(net, from_bus, to_bus, length_km=0.20, std_type="UG2", name="Line I1-I2")

17

Add lines (LV) -> Commercial

In [11]:
for i in range(2, 10):
    from_bus = pp.get_element_index(net, "bus", "Bus C{} LV".format(i-1))
    to_bus = pp.get_element_index(net, "bus", "Bus C{} LV".format(i))
    pp.create_line(net, from_bus, to_bus, length_km=0.030, std_type="OH1", name="Line C{}-C{}".format(i-1, i))

for u, v in [(3, 10), (10, 11), (11, 12), (11, 13), (10, 14), (5, 15), (15, 16), (15, 17), (16, 18), (8, 19), (9, 20)]:
    from_bus = pp.get_element_index(net, "bus", "Bus C{} LV".format(u))
    to_bus = pp.get_element_index(net, "bus", "Bus C{} LV".format(v))
    pp.create_line(net, from_bus, to_bus, length_km=0.030, std_type="OH2", name="Line C{}-C{}".format(u, v))

Add transformer (MV to LV) -> Residential

In [12]:
hv_bus = pp.get_element_index(net, "bus", "Bus R0 MV")
lv_bus = pp.get_element_index(net, "bus", "Bus R1 LV")
pp.create_transformer(net, hv_bus=hv_bus, lv_bus=lv_bus, std_type="0.63 MVA 20/0.4 kV", name="Tr R0-R1")

0

Add transformer (MV to LV) -> Industrial

In [13]:
hv_bus = pp.get_element_index(net, "bus", "Bus I0 MV")
lv_bus = pp.get_element_index(net, "bus", "Bus I1 LV")
pp.create_transformer(net, hv_bus=hv_bus, lv_bus=lv_bus, std_type="0.25 MVA 20/0.4 kV", name="Tr I0-I1")

1

Add transformer (MV to LV) -> Commercial

In [14]:
hv_bus = pp.get_element_index(net, "bus", "Bus C0 MV")
lv_bus = pp.get_element_index(net, "bus", "Bus C1 LV")
pp.create_transformer(net, hv_bus=hv_bus, lv_bus=lv_bus, std_type="0.4 MVA 20/0.4 kV", name="Tr C0-C1")

2

Add switches

In [15]:
bus = pp.get_element_index(net, "bus", "Bus 0 MV")

e1 = pp.get_element_index(net, "bus", "Bus R0 MV")
pp.create_switch(net, bus, e1, et="b", closed=True, type="CB", name="S1")

e2 = pp.get_element_index(net, "bus", "Bus I0 MV")
pp.create_switch(net, bus, e2, et="b", closed=True, type="CB", name="S2")

e3 = pp.get_element_index(net, "bus", "Bus C0 MV")
pp.create_switch(net, bus, e3, et="b", closed=True, type="CB", name="S3")

2

Add external grid connection

In [16]:
bus = pp.get_element_index(net, "bus", "Bus 0 MV")
pp.create_ext_grid(net, bus, name="Feeder")

0

## Define consumption points
Set loads -> Residential

In [17]:
load_buses_r = ["R1", "R11", "R15", "R16", "R17", "R18"]
for b in load_buses_r:
    bus = pp.get_element_index(net, "bus", "Bus {} LV".format(b))
    pp.create_load(net, bus=bus, p_kw=10.0, q_kvar=1.0, scaling=1.0, name="Load{}".format(b))

Set loads -> Industrial

In [18]:
bus = pp.get_element_index(net, "bus", "Bus I2 LV")
pp.create_load(net, bus=bus, p_kw=10.0, q_kvar=1.0, scaling=1.0, name="LoadI2")

6

Set loads -> Commercial

In [19]:
load_buses_c = ["C1", "C12", "C13", "C14", "C17", "C18", "C19", "C20"]
for b in load_buses_c:
    bus = pp.get_element_index(net, "bus", "Bus {} LV".format(b))
    pp.create_load(net, bus=bus, p_kw=10.0, q_kvar=1.0, scaling=1.0, name="Load{}".format(b))

## Export to `.json` files

In [20]:
import os
data_folder = "PowerGridData"

net.bus.to_json(os.path.join(data_folder, "bus.json"))
net.bus_geodata.to_json(os.path.join(data_folder, "bus_geodata.json"))
net.line.to_json(os.path.join(data_folder, "line.json"))
net.trafo.to_json(os.path.join(data_folder, "trafo.json"))
net.switch.to_json(os.path.join(data_folder, "switch.json"))
net.ext_grid.to_json(os.path.join(data_folder, "ext_grid.json"))
net.load.to_json(os.path.join(data_folder, "load.json"))

In [21]:
net.load

,name,bus,p_kw,q_kvar,const_z_percent,const_i_percent,sn_kva,scaling,in_service,type
0,LoadR1,2,10.0,1.0,0.0,0.0,NaN,1.0,True,None
1,LoadR11,12,10.0,1.0,0.0,0.0,NaN,1.0,True,None
2,LoadR15,16,10.0,1.0,0.0,0.0,NaN,1.0,True,None
3,LoadR16,17,10.0,1.0,0.0,0.0,NaN,1.0,True,None
4,LoadR17,18,10.0,1.0,0.0,0.0,NaN,1.0,True,None
5,LoadR18,19,10.0,1.0,0.0,0.0,NaN,1.0,True,None
6,LoadI2,22,10.0,1.0,0.0,0.0,NaN,1.0,True,None
7,LoadC1,24,10.0,1.0,0.0,0.0,NaN,1.0,True,None
8,LoadC12,35,10.0,1.0,0.0,0.0,NaN,1.0,True,None
9,LoadC13,36,10.0,1.0,0.0,0.0,NaN,1.0,True,None


In [22]:
attr = "Load/LoadR1/q_var"
attr.split("/")

['Load', 'LoadR1', 'q_var']

In [24]:
pp.runpp(net, numba=False)

In [26]:
net.ext_grid

,name,bus,vm_pu,va_degree,in_service
0,Feeder,0,1.0,0.0,True


In [27]:
net.res_ext_grid

,p_kw,q_kvar
0,-156.735703,-17.59711


In [30]:
pp.get_element_index(net, "ext_grid", "Feeder")

0

In [36]:
attr = "ext_grid/Feeder/p_kw"
table, name, col = attr.split("/")
table, name, col

('ext_grid', 'Feeder', 'p_kw')

In [41]:
idx = pp.get_element_index(net, table, name)
df = getattr(net, "res_"+table)
df.loc[idx, col]

-156.73570329316644